# Demo: Serenity VaR Calculation API

In addition to supporting risk attribution, the latest version of Serenity also supports running VaR calculations
and backtesting those VaR models with your portfolio. This notebook will show you how to select a VaR model and
use it to compute Value at Risk (VaR) for a portfolio constructed from the Serenity security master.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
%run -i init_demo.py

To run a VaR calculation the first thing we need is a portfolio:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'BTC': 100,
    'ETH': 1000,
    'SOL': 10000,
    'BNB': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

The next thing we need is a model configuration:

In [ ]:
from serenity_sdk.types.common import CalculationContext

model_short_name = 'risk.var.parametric.normal'  
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

Now that we have it we can create the `CalculationContext`, the same as with risk attribution. We will set an as-of-date and pass our parametric VaR model configuration ID. We then run VaR for a given date:

In [ ]:
from datetime import date

# construct the input parameters for VaR
ctx = CalculationContext(as_of_date=date(2020, 5, 1), model_config_id=model_config_id)

# run the VaR calc
result = api.risk().compute_var(ctx, portfolio)
result

We can also run a backtest:

In [ ]:
bt_quantiles = [1, 5, 95, 99]
result = api.risk().compute_var_backtest(ctx, portfolio, date(2020, 5, 1), date(2022, 5, 1), quantiles=bt_quantiles)

In [ ]:
from serenity_sdk.renderers.table import VaRBacktestTables

breach_count_period = 365
var_tables = VaRBacktestTables(result, bt_quantiles, breach_count_period)

baselines = var_tables.get_baselines()
vars_abs_by_qs = var_tables.get_absolute_var_by_quantiles()
vars_rel_by_qs = var_tables.get_relative_var_by_quantiles()
pnls_abs = var_tables.get_absolute_pnl()
pnls_rel = var_tables.get_relative_pnl()
var_breaches = var_tables.get_var_breaches()
rolling_breaches = var_tables.get_rolling_breaches()

# Summarize VaR Backtest

We can see all the breaches over this period:

In [ ]:
breaches_df = var_tables.get_breaches_summary()
display(breaches_df)

Now let's do some charting with the backtest data; this is a sneak peek at the kinds of visualizations we will be offering in the Serenity front-end at the end of 2022:

In [ ]:
from serenity_sdk.renderers.plot import create_traffic_light_report

create_traffic_light_report(var_tables)